# Debug online endpoints locally in Visual Studio Code

In [ ]:
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    CodeConfiguration,
    Environment,
)
from azure.identity import DefaultAzureCredential

## Conectar a workspace

In [ ]:
ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

Found the config file in: /config.json
Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


Conectado al Workspace: naturgyml4


## Definir y crear un endpoint local

In [51]:
local_endpoint_name = "my-local-diabetes-endpoint"

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=local_endpoint_name,
    description="Local endpoint for MLflow diabetes model",
    auth_mode="key",
)

In [52]:
# crear el endpoint
ml_client.begin_create_or_update(endpoint, local=True)

Creating local endpoint (my-local-diabetes-endpoint) Done (0m 0s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'my-local-diabetes-endpoint', 'description': 'Local endpoint for MLflow diabetes model', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/my-local-diabetes-endpoint'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x778fa97ff3d0>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

In [53]:
# obtener el endpoint
ml_client.online_endpoints.get(name=local_endpoint_name, local=True)

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'my-local-diabetes-endpoint', 'description': 'Local endpoint for MLflow diabetes model', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/my-local-diabetes-endpoint'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x778fa97febc0>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {}, 'mirror_traffic': {}, 'kind': None})

## Crear el deployment localmente

In [54]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=local_endpoint_name,
    model=Model(path="./model-1/model/sklearn_regression_model.pkl"),
    code_configuration=CodeConfiguration(
        code="./model-1/onlinescoring", scoring_script="score.py"
    ),
    environment=Environment(
        conda_file="./model-1/environment/conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    ),
    instance_type="STANDARD_DS1_V2",
    instance_count=1,
)

In [55]:

deployment = ml_client.online_deployments.begin_create_or_update(
    deployment, local=True, vscode_debug=True
).result()

Creating local deployment (my-local-diabetes-endpoint / blue) .
Building Docker image from Dockerfile
Step 1/7 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04
 ---> 83631454f85b
Step 2/7 : RUN mkdir -p /var/azureml-app/
 ---> Using cache
 ---> 6c2fc49e6613
Step 3/7 : WORKDIR /var/azureml-app/
 ---> Running in 6347264c3871
 ---> 7590ca79628f
Step 4/7 : COPY conda.yml /var/azureml-app/
 ---> 13e16755c045
Step 5/7 : RUN conda env create -n inf-conda-env --file conda.yml
 ---> Running in ab05ace8dae6
.Retrieving notices: done
Channels:
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: ...working... done


==> WARNING: A newer version of conda exists. <==
    current version: 25.1.1
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



python-3.9.22        | 22.5 MB   |            |   0% 
scipy-1.10.1         | 14.4 MB   |            |   0% 



numpy-1.23.5         | 5.6 MB    |            |   0% 



libopen

LocalEndpointNotFoundError: Local deployment (my-local-diabetes-endpoint / blue) does not exist.

In [61]:
# blue deployment takes 100 traffic
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint, local=True)

Updating local endpoint (my-local-diabetes-endpoint) Done (0m 0s)


ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'my-local-diabetes-endpoint', 'description': 'Local endpoint for MLflow diabetes model', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/my-local-diabetes-endpoint'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x778fa97a9de0>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {'blue': 100}, 'mirror_traffic': {}, 'kind': None})

## Obtener detalles de un endpoint

In [62]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=local_endpoint_name, local=True)
endpoint

ManagedOnlineEndpoint({'public_network_access': None, 'provisioning_state': None, 'scoring_uri': None, 'openapi_uri': None, 'name': 'my-local-diabetes-endpoint', 'description': 'Local endpoint for MLflow diabetes model', 'tags': {}, 'properties': {}, 'print_as_yaml': False, 'id': None, 'Resource__source_path': '', 'base_path': PosixPath('/home/azureuser/.azureml/inferencing/my-local-diabetes-endpoint'), 'creation_context': None, 'serialize': <msrest.serialization.Serializer object at 0x778fa97aa830>, 'auth_mode': 'key', 'location': None, 'identity': None, 'traffic': {'blue': 100}, 'mirror_traffic': {}, 'kind': None})

## 2. Debug your endpoint

In [63]:
endpoint = ml_client.online_endpoints.get(name=local_endpoint_name, local=True)

request_file_path = "./model-1/sample-request.json"

ml_client.online_endpoints.invoke(local_endpoint_name, request_file_path, local=True)

TypeError: OnlineEndpointOperations.invoke() takes 2 positional arguments but 3 positional arguments (and 1 keyword-only argument) were given

In [ ]:
print(endpoint)

## 3. Edit your endpoint

In [ ]:
new_deployment = ManagedOnlineDeployment(
    name="green",
    endpoint_name=endpoint_name,
    model=Model(path="../model-2/model/sklearn_regression_model.pkl"),
    code_configuration=CodeConfiguration(
        code="../model-2/onlinescoring", scoring_script="score.py"
    ),
    environment=Environment(
        conda_file="../model-2/environment/conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=2,
)

deployment = ml_client.online_deployments.begin_create_or_update(
    new_deployment, local=True, vscode_debug=True
).result()

## Borrar el endpoint y deployment

In [ ]:
ml_client.online_endpoints.begin_delete(name=local_endpoint_name, local=True)

In [ ]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=local_endpoint_name, local=True)